# Site Assessment Audit:
### outputs:
* Central Dixie Fire SA Audit
* Tetra Forms APN counts SA central
* SA APN counts central-Power BI
* SA APN counts central-Share Point


## Downloads needed:
* 2 Smartsheets both Central and South or BOP and ANVIL
* 1- Tetrat forms: Site survey form
* Power BI SA tracker
* Site Assessment tracker - Sharepoint


# Libraires

In [1]:
import pandas as pd
import glob
import numpy as np
import os
import smartsheet
import datetime

# Variables

In [2]:
csv = '.csv'
excel = '.xlsx'

In [3]:
while True:
    user = input('What project are we Auditing?: Central or Northern ').lower()
    try:

        #central Set up
        if user == 'central':
            print('Central Audit Started')
            # Counties:
            county= ['ALPINE', 'EL DORADO', 'LAKE', 'NEVADA', 'PLACER', 'TUOLUMNE', 'MENDOCINO', 'TULARE', 'KERN']

            # Division. * northern goes here becasue we want everything not in northern Division
            division = 'Northern Division'

            # smartsheets Shet IDs
            sheet_id1 = '1844440261257092' # central
            sheet_id2= '6292262490531716' # south central

            # Central's API
            API1 = smartsheet.Smartsheet('6jJFxnAfFsDErU0BSr1Ae4D6cHq0zenczVR12') # API central

            API1.errors_as_exceptions(True)

            # Central's API
            API2 = smartsheet.Smartsheet("H8mqAo60J2zGHSrJEwmlMoY88A9yjgCo6RnSI") # API south central
            API2.errors_as_exceptions(True)
            break

        #northern Set up
        elif user == 'northern':
            print('Northern Audit Started')
            # Counties:
            county = ['LASSEN', 'PLUMAS', 'SISKIYOU', 'TRINITY', 'SHASTA', 'TEHAMA']

            # Division. * northern goes here becasue we want everything not in northern Division
            division = 'Central Division'

            # smartsheets Shet IDs
            sheet_id1 = '3589680131270532' # BOP
            sheet_id2= '5367491607193476' # ANVIL

            # Central's API
            API1 = smartsheet.Smartsheet('6w7vusWNdvdm9TJmjImZfbh0SPQgi3HJaekNb') # API BOP

            API1.errors_as_exceptions(True)

            # Central's API
            API2 = smartsheet.Smartsheet("MQQmCjvvFWLKBoX9K7A33rHSoLnuj1MGVJuVK") # API ANVIL
            API2.errors_as_exceptions(True)
            break
    except ValueError:
        print("\ninvaild input Only 'Central or Northern' allowed")

What project are we Auditing?: Central or Northern aja
What project are we Auditing?: Central or Northern lakjhgf
What project are we Auditing?: Central or Northern northern
Northern Audit Started


# Function to set up Smartsheet into a readable Database

In [4]:
def smartsheet_loadup(sheet):
    columns = [col.title for col in sheet.columns]
    rows = []
    for row in sheet.rows:
        cells = []
        for cell in row.cells:
            cells.append(cell.value)
        rows.append(cells)
    df = pd.DataFrame(rows, columns=columns)
    return df

Area1_SS = smartsheet_loadup(API1.Sheets.get_sheet(sheet_id1))
Area2_SS = smartsheet_loadup(API2.Sheets.get_sheet(sheet_id2))

df_both = pd.concat([Area1_SS,Area2_SS])

# Share point

In [5]:
SA_sharepoint_path = r"C:\Users\jacque.trahan\Downloads\Site Assessment Tracker*"
Share_load = glob.glob(SA_sharepoint_path + csv)
sa_share = pd.read_csv(max(Share_load, key=os.path.getctime))

# Power Bi (APN Counts only)

In [6]:
sa_powerBi_path = r"C:\Users\jacque.trahan\Downloads\SA Tracker-Power BI*"
bi_load = glob.glob(sa_powerBi_path + csv)
power_bi_sa = pd.read_csv(max(bi_load, key=os.path.getctime))

In [7]:
sa_bi = power_bi_sa.copy()

# Tetra forms

In [8]:
df3 =pd.read_excel("../../tetra forms ASB and SA/SA_TetraForms.xlsx", parse_dates=True)

In [9]:
smartsheets = df_both.copy()
sa = sa_share.copy()
tetarforms = df3.copy()

In [10]:
tetarforms = tetarforms[tetarforms['record_void'] != "Yes"]

# Site Assessment Cleaning

In [11]:
# sa column selection
sa = sa[['APN','Date Completed','Automobiles', 'Motorcycle', 'ATV/UTV/Snowmobiles',
       'Ag/Commercial/Construction', 'RV', 'Trailers', 'Vessel', 'Chimney','Division']]

In [12]:
# only use central division APNs and parcel information
sa = sa[~sa['Division'].isin([division])]

### Get the value counts of APNS in Sa Share Point

In [13]:
# share point
sa['APN'].value_counts().to_excel('SA APN counts central-Share Point.xlsx')

# power bi
sa_bi['APN'].value_counts().to_excel('SA APN counts central-Power BI.xlsx')

In [14]:
# replace all 0s to No and everything greater than 0 = 'yes'
sa['Chimney'] = np.where(sa['Chimney'] > 0, 'Yes', 'No')

In [15]:
# set up date columns
sa['Date Completed'] = pd.to_datetime(sa['Date Completed']).dt.date

In [16]:
sa.rename(columns={'Date Completed': 'SA Survey Date','APN': 'APN_SA','Motorcycle':'Motorcycle_SA',
                  'RV': 'RV_SA', 'Trailers':'Trailers_SA', 'Vessel': 'Vessel_SA'}, inplace=True)

In [17]:
# set up numerical values for al vehicles
sa['Automobiles'] = pd.to_numeric(sa['Automobiles'])
sa['Motorcycle_SA'] = pd.to_numeric(sa['Motorcycle_SA'])
sa['ATV/UTV/Snowmobiles'] = pd.to_numeric(sa['ATV/UTV/Snowmobiles'])
sa['Ag/Commercial/Construction'] = pd.to_numeric(sa['Ag/Commercial/Construction'])
sa['RV_SA'] = pd.to_numeric(sa['RV_SA'])
sa['Trailers_SA'] = pd.to_numeric(sa['Trailers_SA'])
sa['Vessel_SA'] = pd.to_numeric(sa['Vessel_SA'])

In [18]:
sa.fillna({'SA Survey Date':0,
           'Automobiles': 0,
           'Motorcycle_SA': 0,
           'ATV/UTV/Snowmobiles': 0,
           'Ag/Commercial/Construction': 0,
           'RV_SA': 0,
           'Trailers_SA': 0,
           'Vessel_SA': 0}, inplace=True)

In [19]:
sa.columns = map(str.upper, sa.columns)

# Tetra Forms Cleaning

In [20]:
# get just the counties we need
tetarforms = tetarforms[tetarforms['county'].isin(county)]

### Get the value counts of APNS in Tetra Forms for central

In [21]:
tetarforms['apn'].value_counts().to_excel('Tetra Forms APN counts SA central.xlsx')

In [22]:
tetarforms[(tetarforms['survey_date'] == "11/9/2021; 11/18/2021")] = 11/18/2021

In [23]:
# format dates 
tetarforms['survey_date'] = pd.to_datetime(tetarforms['survey_date']).dt.date

In [24]:
#get columsn we need
tetarforms = tetarforms[['apn','survey_date','chimney_quantity', 'automobile_quantity', 'motorcycle_quantity',
                         'atv_utv_quantity', 'comm_equip_quantity', 'rvs_quantity', 'trailer_quantity', 'vessel_quantity']]

In [25]:
tetarforms.fillna({'survey_date':0,
           'automobile_quantity': 0,
           'motorcycle_quantity': 0,
           'atv_utv_quantity': 0,
           'comm_equip_quantity': 0,
           'rvs_quantity': 0,
           'trailer_quantity': 0,
           'vessel_quantity': 0}, inplace=True)

In [26]:
tetarforms['chimney_quantity'] = np.where(tetarforms['chimney_quantity'] > 0, 'Yes', 'No')

In [27]:
tetarforms.rename(columns={'apn': 'APN_TF', 'motorcycle_quantity':'motorcycle_quantity_TF'}, inplace=True)

In [28]:
tetarforms.columns = map(str.upper, tetarforms.columns)

# TF_SA_Merge -  tetraforms and SA

In [29]:

TF_SA_Merge = sa.merge(tetarforms,
                 left_on='APN_SA',
                 right_on='APN_TF',
                 how='outer',
                 suffixes=('_SA', '_TF'))

# Smart sheet Cleaning

In [30]:
# smart sheet column selection
smartsheets = smartsheets[['APN_ROW Segment','Street #','Street Name', 'Structural Status','County',
                           'Chimney',
                           'Site Assessment',
                          'Number of Passenger Vehicles', 'Number of ATV_UTV_snowmobiles',
                          'Number of Ag_Commercial_Construction Equipment','Number of Motorcycles',
                          'Number of RV\'s', 'Number of Trailers', 'Number of Vessels (Boat_Jetski)', 'Total Number of Vehicles']]

smartsheets = smartsheets.copy()

# take of the time on the dates
smartsheets['Site Assessment'] = pd.to_datetime(smartsheets['Site Assessment']).dt.date

In [31]:
# fill N/A or blanks as needed 
smartsheets.fillna({'Chimney': 'No',
                    'Site Assessment': 0,
                    'Number of Passenger Vehicles': 0,
                    'Number of ATV_UTV_snowmobiles': 0,
                    'Number of Ag_Commercial_Construction Equipment': 0,
                    'Number of Motorcycles': 0,
                    'Number of RV\'s':0,
                    'Number of Trailers': 0,
                    'Number of Vessels (Boat_Jetski)': 0}, inplace=True)

In [32]:
# Rename columns
smartsheets.rename(columns={'County':'County_SS','Site Assessment': 'SA Survey Date', 'Number of Passenger Vehicles': 'Automobiles',
                           'Number of ATV_UTV_snowmobiles': 'ATV/UTV/Snowmobiles',
                           'Number of Ag_Commercial_Construction Equipment': 'Ag/Commercial/Construction',
                           'Number of Motorcycles': 'Motorcycles_SS', 'Number of RV\'s': 'RV_SS',
                           'Number of Trailers':'Trailers_SS','Number of Vessels (Boat_Jetski)':'Vessel_SS'}, inplace=True)

In [33]:
# capitalize all column headers
smartsheets.columns = map(str.upper, smartsheets.columns)

# Merge Smartsheets with TF and SA' merged database

In [34]:
# merge both data frames for QC
final = smartsheets.merge(TF_SA_Merge,
                         left_on='APN_ROW SEGMENT',
                         right_on='APN_SA',
                         how='left',
                         suffixes=('_SS', '_SA'))

In [35]:
# column orinizationa
final = final[['APN_ROW SEGMENT','APN_SA', 'APN_TF','STREET #','STREET NAME','STRUCTURAL STATUS','COUNTY_SS',
               'SA SURVEY DATE_SS', 'SA SURVEY DATE_SA','SURVEY_DATE',
               'CHIMNEY_SS','CHIMNEY_SA','CHIMNEY_QUANTITY',
               'AUTOMOBILES_SS', 'AUTOMOBILES_SA','AUTOMOBILE_QUANTITY',
               'MOTORCYCLES_SS', 'MOTORCYCLE_SA', 'MOTORCYCLE_QUANTITY_TF',
              'ATV/UTV/SNOWMOBILES_SS', 'ATV/UTV/SNOWMOBILES_SA','ATV_UTV_QUANTITY',
              'AG/COMMERCIAL/CONSTRUCTION_SS', 'AG/COMMERCIAL/CONSTRUCTION_SA','COMM_EQUIP_QUANTITY',
              'RV_SS', 'RV_SA','RVS_QUANTITY',
              'TRAILERS_SS', 'TRAILERS_SA','TRAILER_QUANTITY',
              'VESSEL_SS','VESSEL_SA','VESSEL_QUANTITY',
               'DIVISION','TOTAL NUMBER OF VEHICLES']]

In [36]:
final['SA SURVEY DATE_SS'] = pd.to_datetime(final['SA SURVEY DATE_SS'], errors='coerce').dt.date
final['SA SURVEY DATE_SA'] = pd.to_datetime(final['SA SURVEY DATE_SA'], errors='coerce').dt.date
final['SURVEY_DATE'] = pd.to_datetime(final['SURVEY_DATE'], errors='coerce').dt.date

# add 0's to any remaining blanks

In [37]:
# This will tell us that we have no data in the remander of the cells
final.fillna(0,inplace=True)

# compare function for each category

In [38]:
def compare_cols(df,comp1,comp2,comp3):
    if (df[comp1] == df[comp2]) & (df[comp1] == df[comp3]):
        return 1
    else:
        return 0

# Survey date checks

In [39]:
final.insert(final.columns.get_loc('SURVEY_DATE')+1, "Survey Matches",
            final.apply(compare_cols, comp1='SA SURVEY DATE_SS', comp2='SA SURVEY DATE_SA', comp3='SURVEY_DATE', axis=1))

# CHIMNEY

In [40]:
final.insert(final.columns.get_loc('CHIMNEY_QUANTITY')+1, "Chimney Macthes",
            final.apply(compare_cols, comp1='CHIMNEY_SS', comp2='CHIMNEY_SA', comp3='CHIMNEY_QUANTITY', axis=1))

# AUTOMOBILES

In [41]:
final.insert(final.columns.get_loc('AUTOMOBILE_QUANTITY')+1, "AUTOMOBILE-M",
            final.apply(compare_cols, comp1='AUTOMOBILES_SS', comp2='AUTOMOBILES_SA', comp3='AUTOMOBILE_QUANTITY', axis=1))

# MOTORCYCLES

In [42]:
final.insert(final.columns.get_loc('MOTORCYCLE_QUANTITY_TF')+1, "Motorcycle-M",
            final.apply(compare_cols, comp1='MOTORCYCLES_SS', comp2='MOTORCYCLE_SA', comp3='MOTORCYCLE_QUANTITY_TF', axis=1))

# ATV/UTV/SNOWMOBILES

In [43]:
final.insert(final.columns.get_loc('ATV_UTV_QUANTITY')+1, "ATV-M",
            final.apply(compare_cols,
                        comp1='ATV/UTV/SNOWMOBILES_SS', comp2='ATV/UTV/SNOWMOBILES_SA', comp3='ATV_UTV_QUANTITY', axis=1))

# AG/COMMERCIAL/CONSTRUCTION

In [44]:
final.insert(final.columns.get_loc('COMM_EQUIP_QUANTITY')+1, "COMM-M",
            final.apply(compare_cols,
                        comp1='AG/COMMERCIAL/CONSTRUCTION_SS',
                        comp2='AG/COMMERCIAL/CONSTRUCTION_SA', comp3='COMM_EQUIP_QUANTITY', axis=1))

# RV

In [45]:
final.insert(final.columns.get_loc('RVS_QUANTITY')+1, "RV-M",
            final.apply(compare_cols, comp1='RV_SS', comp2='RV_SA', comp3='RVS_QUANTITY', axis=1))

# TRAILERS

In [46]:
final.insert(final.columns.get_loc('TRAILER_QUANTITY')+1, "Trailer-M",
            final.apply(compare_cols, comp1='TRAILERS_SS', comp2='TRAILERS_SA', comp3='TRAILER_QUANTITY', axis=1))

# VESSEL

In [47]:
final.insert(final.columns.get_loc('VESSEL_QUANTITY')+1, "Vessel-M",
            final.apply(compare_cols, comp1='VESSEL_SS', comp2='VESSEL_SA', comp3='VESSEL_QUANTITY', axis=1))

# color in the compare columns

In [48]:
final_style = final.style.set_properties(**{'background-color': 'lightblue'},
                          subset=['Survey Matches', 'Chimney Macthes','AUTOMOBILE-M','Motorcycle-M',"ATV-M",
                                  'COMM-M',"RV-M",'Trailer-M' ,'Vessel-M'])

In [49]:
final_style.to_excel('Central Dixie Fire SA Audit.xlsx')